In [1]:
# Data science
import pandas as pd

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [2]:
# Read the CSV files
parks_df = pd.read_csv('Resources/parks.csv')
species_df = pd.read_csv('Resources/species.csv')

In [3]:
parks_df.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [4]:
species_df.head()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN


In [5]:
#drop_list = ["Record Status", "Occurrence", "Nativeness", "Abundance", "Seasonality"]
species_df.drop(["Record Status", "Occurrence", "Nativeness", "Abundance", "Seasonality"], axis=1, inplace=True)

In [6]:
species_df.head()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Conservation Status
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Species of Concern
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Endangered
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",NaN


In [7]:
merged_df = pd.merge(parks_df, species_df, on="Park Name")
merged_df.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude,Species ID,Category,Order,Family,Scientific Name,Common Names,Conservation Status
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1000,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,NaN
1,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1001,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",NaN
2,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1002,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Species of Concern
3,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1003,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Endangered
4,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1004,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",NaN


In [8]:
df_split = merged_df.assign(State=merged_df["State"].str.split(", ")).explode("State")

df_split = df_split.reset_index(drop=True)

df_split.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude,Species ID,Category,Order,Family,Scientific Name,Common Names,Conservation Status
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1000,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,NaN
1,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1001,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",NaN
2,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1002,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Species of Concern
3,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1003,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Endangered
4,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1004,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",NaN


In [9]:
df_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138242 entries, 0 to 138241
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Park Code            138242 non-null  object 
 1   Park Name            138242 non-null  object 
 2   State                138242 non-null  object 
 3   Acres                138242 non-null  int64  
 4   Latitude             138242 non-null  float64
 5   Longitude            138242 non-null  float64
 6   Species ID           138242 non-null  object 
 7   Category             138242 non-null  object 
 8   Order                136478 non-null  object 
 9   Family               136455 non-null  object 
 10  Scientific Name      138242 non-null  object 
 11  Common Names         102299 non-null  object 
 12  Conservation Status  5294 non-null    object 
dtypes: float64(2), int64(1), object(10)
memory usage: 13.7+ MB


In [10]:
print(df_split["Park Name"].nunique())
df_split["Park Name"].unique()

56


array(['Acadia National Park', 'Arches National Park',
       'Badlands National Park', 'Big Bend National Park',
       'Biscayne National Park',
       'Black Canyon of the Gunnison National Park',
       'Bryce Canyon National Park', 'Canyonlands National Park',
       'Capitol Reef National Park', 'Carlsbad Caverns National Park',
       'Channel Islands National Park', 'Congaree National Park',
       'Crater Lake National Park', 'Cuyahoga Valley National Park',
       'Denali National Park and Preserve', 'Death Valley National Park',
       'Dry Tortugas National Park', 'Everglades National Park',
       'Gates Of The Arctic National Park and Preserve',
       'Glacier National Park', 'Glacier Bay National Park and Preserve',
       'Great Basin National Park', 'Grand Canyon National Park',
       'Great Sand Dunes National Park and Preserve',
       'Great Smoky Mountains National Park', 'Grand Teton National Park',
       'Guadalupe Mountains National Park', 'Haleakala National

In [11]:
# Confirm that states are split
mask = df_split.loc[df_split["Park Name"] == "Yellowstone National Park"]
print(mask)

       Park Code                  Park Name State    Acres  Latitude  \
122460      YELL  Yellowstone National Park    WY  2219791      44.6   
122461      YELL  Yellowstone National Park    MT  2219791      44.6   
122462      YELL  Yellowstone National Park    ID  2219791      44.6   
122463      YELL  Yellowstone National Park    WY  2219791      44.6   
122464      YELL  Yellowstone National Park    MT  2219791      44.6   
...          ...                        ...   ...      ...       ...   
134353      YELL  Yellowstone National Park    MT  2219791      44.6   
134354      YELL  Yellowstone National Park    ID  2219791      44.6   
134355      YELL  Yellowstone National Park    WY  2219791      44.6   
134356      YELL  Yellowstone National Park    MT  2219791      44.6   
134357      YELL  Yellowstone National Park    ID  2219791      44.6   

        Longitude Species ID Category             Order             Family  \
122460     -110.5  YELL-1000   Mammal      Artiodactyla  

In [15]:
# Create engine for conection to the SQLite
engine = create_engine('sqlite:///national_parks_all.sqlite')

# Save the dataframes to the database
parks_df.to_sql('parks', engine, index=False, if_exists='replace')
species_df.to_sql('species', engine, index=False, if_exists='replace')
df_split.to_sql('all', engine, index=False, if_exists='replace')

138242

In [ ]:
# Read the data from the database
# with engine.connect() as connection:
#     parks_table = pd.read_sql('SELECT * FROM parks', connection)
#     species_table = pd.read_sql('SELECT * FROM species', connection)

# # species_table.head()

In [ ]:
# parks_table.head()

In [ ]:
# # Split parks with multiple states into duplicate rows with one for each state

# query = """
#     WITH split_states AS (
#         SELECT
#             p."Park Code",
#             p."Park Name",
#             TRIM(CASE
#                 WHEN INSTR(p.State, ', ') > 0 THEN SUBSTR(p.State, 1, INSTR(p.State, ', ') - 1)
#                 ELSE p.State
#             END) AS State,
#             p.Acres,
#             p.Latitude,
#             p.Longitude
#         FROM
#             parks AS p
#         UNION
#         SELECT
#             p."Park Code",
#             p."Park Name",
#             TRIM(CASE
#                 WHEN INSTR(p.State, ', ') > 0 THEN SUBSTR(p.State, INSTR(p.State, ', ') + 1)
#                 ELSE ''
#             END) AS State,
#             p.Acres,
#             p.Latitude,
#             p.Longitude
#         FROM
#             parks AS p
#         UNION
#         SELECT
#             p."Park Code",
#             p."Park Name",
#             TRIM(CASE
#                 WHEN INSTR(p.State, ', ') > 0 THEN SUBSTR(p.State, INSTR(p.State, ', ') + 2)
#                 ELSE ''
#             END) AS State,
#             p.Acres,
#             p.Latitude,
#             p.Longitude
#         FROM
#             parks AS p
#     )
#     SELECT
#         s."Species ID",
#         s.Category,
#         s."Order",
#         s.Family,
#         s."Scientific Name",
#         s."Common Names",
#         s."Conservation Status",
#         t."Park Code",
#         t."Park Name",
#         TRIM(t.State) AS State,
#         t.Acres,
#         t.Latitude,
#         t.Longitude
#     FROM
#         split_states as t
#     JOIN
#         species as s ON s."Park Name" = t."Park Name"
#     WHERE
#         s."Conservation Status" <> 'None'
#         AND TRIM(t.State) <> '';
#     """


# combined_df = pd.read_sql(text(query), con=engine)
# combined_df

In [ ]:
# # Confirm that states are split
# combined_df.loc[combined_df["Park Name"] == "Death Valley National Park"].reset_index(drop=True)

In [ ]:
# combined_df.to_sql('combined', engine, index=False, method='multi', if_exists='replace')

# engine = create_engine('sqlite:///national_parks_3.sqlite')

# with engine.connect() as connection:
#     combined_table = pd.read_sql('SELECT * FROM combined', connection)

In [16]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

all
-----------
Park Code TEXT
Park Name TEXT
State TEXT
Acres BIGINT
Latitude FLOAT
Longitude FLOAT
Species ID TEXT
Category TEXT
Order TEXT
Family TEXT
Scientific Name TEXT
Common Names TEXT
Conservation Status TEXT

parks
-----------
Park Code TEXT
Park Name TEXT
State TEXT
Acres BIGINT
Latitude FLOAT
Longitude FLOAT

species
-----------
Species ID TEXT
Park Name TEXT
Category TEXT
Order TEXT
Family TEXT
Scientific Name TEXT
Common Names TEXT
Conservation Status TEXT



In [17]:
print(df_split["State"].nunique())
df_split.State.sort_values().unique()

27


array(['AK', 'AR', 'AZ', 'CA', 'CO', 'FL', 'HI', 'ID', 'KY', 'ME', 'MI',
       'MN', 'MT', 'NC', 'ND', 'NM', 'NV', 'OH', 'OR', 'SC', 'SD', 'TN',
       'TX', 'UT', 'VA', 'WA', 'WY'], dtype=object)

In [18]:
print(df_split["Conservation Status"].nunique())
df_split["Conservation Status"].unique()

11


array([nan, 'Species of Concern', 'Endangered', 'In Recovery',
       'Threatened', 'Under Review', 'Proposed Threatened', 'Extinct',
       'Proposed Endangered', 'Resident', 'Breeder', 'Migratory'],
      dtype=object)

In [ ]:
# Q3: How does the size of different parks compare with the conservation status of the animals in each park across the country?

In [ ]:
query = """
    SELECT
        "Park Name",
        Latitude,
        Longitude,
        "Conservation Status",
        COUNT ("Conservation Status") AS "Species Count",
        State,
        Acres
    FROM
        combined
    WHERE
        "Conservation Status" IN ('Endangered', 'Threatened', 'Species of Concern', 'Under Review')
    GROUP BY 
        "Park Name", "State", "Conservation Status"
    ORDER BY
        Acres DESC
    """

q3_df = pd.read_sql(text(query), con=engine)
q3_df.head(15)

In [ ]:
q3_df.info()

In [ ]:
q3_df.value_counts("Conservation Status")

In [ ]:
# Total Species count per park
df3 = q3_df.groupby("Park Name")["Species Count"].sum()
df3 = df3.sort_values(ascending=False)
df3

In [ ]:
# df_split = q3_df.assign(State=q3_df["State"].str.split(", ")).explode("State")

# df_split = df_split.reset_index(drop=True)

# df_split.head(20)

In [ ]:
# # Save the dataframes to the database
# q3_df.to_sql('conservation', engine, index=False, if_exists='replace')

In [ ]:
# engine.dispose()